# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

import csv

# Import API key
from api_keys import g_key

In [2]:
# Load the csv exported in Part I to a DataFrame
vacation_data = pd.read_csv('../output_data/City_data.csv', delimiter = ',')
vacation_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mataura,-46.19,168.86,48.99,80,59,14.00,NZ,1603122785
1,1,Fukuechō,32.69,128.84,68.59,69,51,19.98,JP,1603122786
2,2,Vaini,-21.20,-175.20,69.80,77,20,9.17,TO,1603122786
3,3,Punta Arenas,-53.15,-70.92,42.80,61,40,32.21,CL,1603122729
4,4,Buta,2.79,24.73,76.12,92,100,0.72,CD,1603122786


### Humidity Heatmap

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations = vacation_data[['Lat','Lng']]
weight = vacation_data['Humidity']

In [6]:
# Customize the size of the figure
figure_layout = {
    'width': '1000px',
    'height': '350px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig= gmaps.figure(layout=figure_layout, zoom_level = 2.25, center = (28.42,2.294))

# Add Heatmap layer to map that displays humidity for every city in dataset
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=80,
                                 point_radius = 3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='350px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria

In [7]:
# Narrow down the cities to fit weather conditions.
# Criteria: max temp between 70 and 80, wind speed less than 10, 0 cloudiness 
perfect_conditions = vacation_data.loc[(vacation_data['Max Temp']<80) 
                                       & (vacation_data['Max Temp']>70) 
                                       & (vacation_data['Wind Speed']<10)
                                       & (vacation_data['Cloudiness']==0)]
                                          
perfect_conditions

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
64,64,Mehtar Lām,34.67,70.21,72.14,21,0,3.04,AF,1603122792
128,128,Guerrero Negro,27.98,-114.06,76.06,65,0,5.82,MX,1603122288
191,191,Sinjār,36.32,41.88,72.54,21,0,7.02,IQ,1603122800
294,294,Angoche,-16.23,39.91,77.32,74,0,8.93,MZ,1603122893
320,320,Lavumisa,-27.31,31.89,79.83,63,0,6.71,SZ,1603122896
349,349,Betioky,-23.72,44.38,75.85,50,0,6.49,MG,1603122902
369,369,Gweru,-19.45,29.82,78.69,18,0,8.10,ZW,1603122905
399,399,Vicuña,-30.03,-70.71,79.00,20,0,5.01,CL,1603122908
429,429,Tura,25.52,90.22,72.97,92,0,2.13,IN,1603122912
483,483,Pueblo Nuevo,23.38,-105.38,74.01,39,0,2.30,MX,1603122918


### Hotel Map

In [8]:
# Store into variable named hotel_df
hotel_df = perfect_conditions

In [10]:
# Add a "Hotel Name" column to the DataFrame.
hotel_df['Hotel Name'] = ""
hotel_df;

<ipython-input-10-8e4bf1060b54>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


In [11]:
#Set parameters to search for hotels with 5000 meters.
params = {'key': g_key, 
          'radius' : 5000,
          'type' : 'lodging'
         }

# Create function to find nearby hotels  
def findNearby(params): 
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    response = requests.get(base_url, params=params)
    data = response.json()
    return data


In [58]:
# For each city that fits weather conditions, use Google Places API for each city's coodinates to find hotels
for index, row in hotel_df.iterrows():
    params['location']= f" {row['Lat']}, {row['Lng']}"
    data = findNearby(params)

    # Use try/catch to pass on locations without nearby hotels. 
    try: 
        # Store the first Hotel result into the DataFrame.
        hotel_df.loc[index,'Hotel Name'] = data['results'][0]['name']
    except: 
        hotel_df.loc[index,'Hotel Name'] = 'No Hotel Found'

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [59]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
64,64,Mehtar Lām,34.67,70.21,72.14,21,0,3.04,AF,1603122792,سلطان غازی بابا مارکیت
128,128,Guerrero Negro,27.98,-114.06,76.06,65,0,5.82,MX,1603122288,"Casa Laguna, Bed & Breakfast"
191,191,Sinjār,36.32,41.88,72.54,21,0,7.02,IQ,1603122800,زوماني فون 1
294,294,Angoche,-16.23,39.91,77.32,74,0,8.93,MZ,1603122893,Guest House HESADA
320,320,Lavumisa,-27.31,31.89,79.83,63,0,6.71,SZ,1603122896,ROADSIDE B&B
349,349,Betioky,-23.72,44.38,75.85,50,0,6.49,MG,1603122902,Bungalows Acacias Hotel
369,369,Gweru,-19.45,29.82,78.69,18,0,8.10,ZW,1603122905,Suskwe Lodges
399,399,Vicuña,-30.03,-70.71,79.00,20,0,5.01,CL,1603122908,Los Mosaicos
429,429,Tura,25.52,90.22,72.97,92,0,2.13,IN,1603122912,Hotel RIKMAN Continental
483,483,Pueblo Nuevo,23.38,-105.38,74.01,39,0,2.30,MX,1603122918,No Hotel Found


In [60]:
# Code given by starter file 
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [61]:
# Add marker layer ontop of heat map
figure_layout = {
    'width': '1000px',
    'height': '350px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}
fig= gmaps.figure(layout=figure_layout, zoom_level = 2, center = (28.42,2.294))
fig.add_layer(heat_layer)

# Assign the marker layer to a variable
# Plot each pin containing the hotel name, city, country.
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(border='1px solid black', height='350px', margin='0 auto 0 auto', padding='1px', wi…